In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

In [2]:
# Load your dataset
data = pd.read_csv(r"C:\Users\Zimm\Downloads\parkinsons.data")

In [3]:
# Display the first few rows of the dataset
data.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [4]:
data.info

<bound method DataFrame.info of                name  MDVP:Fo(Hz)  MDVP:Fhi(Hz)  MDVP:Flo(Hz)  MDVP:Jitter(%)  \
0    phon_R01_S01_1      119.992       157.302        74.997         0.00784   
1    phon_R01_S01_2      122.400       148.650       113.819         0.00968   
2    phon_R01_S01_3      116.682       131.111       111.555         0.01050   
3    phon_R01_S01_4      116.676       137.871       111.366         0.00997   
4    phon_R01_S01_5      116.014       141.781       110.655         0.01284   
..              ...          ...           ...           ...             ...   
190  phon_R01_S50_2      174.188       230.978        94.261         0.00459   
191  phon_R01_S50_3      209.516       253.017        89.488         0.00564   
192  phon_R01_S50_4      174.688       240.005        74.287         0.01360   
193  phon_R01_S50_5      198.764       396.961        74.904         0.00740   
194  phon_R01_S50_6      214.289       260.277        77.973         0.00567   

     MD

In [5]:
# Check for non-numeric columns
print(data.dtypes)

name                 object
MDVP:Fo(Hz)         float64
MDVP:Fhi(Hz)        float64
MDVP:Flo(Hz)        float64
MDVP:Jitter(%)      float64
MDVP:Jitter(Abs)    float64
MDVP:RAP            float64
MDVP:PPQ            float64
Jitter:DDP          float64
MDVP:Shimmer        float64
MDVP:Shimmer(dB)    float64
Shimmer:APQ3        float64
Shimmer:APQ5        float64
MDVP:APQ            float64
Shimmer:DDA         float64
NHR                 float64
HNR                 float64
status                int64
RPDE                float64
DFA                 float64
spread1             float64
spread2             float64
D2                  float64
PPE                 float64
dtype: object


In [6]:
# Drop the non-numeric 'name' column
data = data.drop(columns=['name'])

In [12]:
# Define features and target
X = data.drop('status', axis=1)
y = data['status']

In [15]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [19]:
# Save the model
joblib.dump(model, 'parkinsons_model.pkl')

['parkinsons_model.pkl']

In [21]:
# Return a confirmation
"Model trained and saved successfully."

'Model trained and saved successfully.'

In [24]:
import os

# Create directories
os.makedirs('parkinsons_detection/app/templates', exist_ok=True)
os.makedirs('parkinsons_detection/static', exist_ok=True)
os.makedirs('parkinsons_detection/model', exist_ok=True)

# Create files
open('parkinsons_detection/app/__init__.py', 'w').close()
open('parkinsons_detection/app/routes.py', 'w').close()
open('parkinsons_detection/app/model.py', 'w').close()
open('parkinsons_detection/app/templates/index.html', 'w').close()
open('parkinsons_detection/run.py', 'w').close()
open('parkinsons_detection/requirements.txt', 'w').close()


In [26]:
with open('parkinsons_detection/app/__init__.py', 'w') as f:
    f.write("""
from flask import Flask

def create_app():
    app = Flask(__name__)
    from .routes import main
    app.register_blueprint(main)
    return app
    """)


In [28]:
with open('parkinsons_detection/app/routes.py', 'w') as f:
    f.write("""
from flask import Blueprint, render_template, request
import joblib
import numpy as np

main = Blueprint('main', __name__)

# Load the trained model
model = joblib.load('model/parkinsons_model.pkl')

@main.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Get form data
        form_data = request.form.to_dict()
        features = [float(value) for value in form_data.values()]
        
        # Predict using the model
        prediction = model.predict([features])
        result = 'Positive' if prediction[0] == 1 else 'Negative'
        
        return render_template('index.html', result=result)
    return render_template('index.html')
    """)


In [30]:
with open('parkinsons_detection/app/templates/index.html', 'w') as f:
    f.write("""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Parkinson's Disease Detection</title>
</head>
<body>
    <h1>Parkinson's Disease Detection</h1>
    <form method="POST">
        <!-- Add form fields for each feature here -->
        <input type="text" name="MDVP:Fo(Hz)" placeholder="MDVP:Fo(Hz)" required>
        <input type="text" name="MDVP:Fhi(Hz)" placeholder="MDVP:Fhi(Hz)" required>
        <input type="text" name="MDVP:Flo(Hz)" placeholder="MDVP:Flo(Hz)" required>
        <!-- Add more fields as needed -->
        <button type="submit">Predict</button>
    </form>
    {% if result %}
        <h2>Result: {{ result }}</h2>
    {% endif %}
</body>
</html>
    """)


In [32]:
with open('parkinsons_detection/run.py', 'w') as f:
    f.write("""
from app import create_app

app = create_app()

if __name__ == '__main__':
    app.run(debug=True)
    """)


In [34]:
with open('parkinsons_detection/requirements.txt', 'w') as f:
    f.write("""
Flask
scikit-learn
pandas
numpy
joblib
    """)


In [36]:
import os
import shutil

# Create directories
os.makedirs('parkinsons_detection/model', exist_ok=True)

# Path to the uploaded model file
source_model_path = r"C:\Users\Zimm\Desktop\CodClause\PAK\parkinsons_model.pkl"

# Destination path
destination_model_path = 'parkinsons_detection/model/parkinsons_model.pkl'

# Copy the model file to the destination path
shutil.copy(source_model_path, destination_model_path)

# Verify if the model file exists in the destination path
if os.path.exists(destination_model_path):
    print(f"Model file successfully copied to {destination_model_path}")
else:
    print("Failed to copy the model file.")


Model file successfully copied to parkinsons_detection/model/parkinsons_model.pkl


In [38]:
# Step 1: Change to the directory containing run.py
%cd C:\Users\Zimm\Desktop\CodClause\PAK\parkinsons_detection


C:\Users\Zimm\Desktop\CodClause\PAK\parkinsons_detection


In [40]:
# Step 2: Verify the current directory
%pwd



'C:\\Users\\Zimm\\Desktop\\CodClause\\PAK\\parkinsons_detection'

In [42]:
# Step 3: List files in the current directory to confirm run.py is there
!dir


 Volume in drive C is Acer
 Volume Serial Number is EC9F-7E59

 Directory of C:\Users\Zimm\Desktop\CodClause\PAK\parkinsons_detection

30-06-2024  18:52    <DIR>          .
30-06-2024  20:59    <DIR>          ..
30-06-2024  19:31    <DIR>          app
30-06-2024  18:55    <DIR>          model
30-06-2024  21:00                50 requirements.txt
30-06-2024  21:00               111 run.py
30-06-2024  18:52    <DIR>          static
               2 File(s)            161 bytes
               5 Dir(s)  46,367,232,000 bytes free


In [ ]:
# Step 4: Run the Flask app
!python run.py